In [1]:
from neo4j import GraphDatabase
from allennlp.predictors.predictor import Predictor
import allennlp_models.rc
import time

driver = GraphDatabase.driver('neo4j://localhost:11003', auth=('chatbot', 'password'))
mrc = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bidaf-elmo-model-2020.03.19.tar.gz")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


In [2]:
from helper import *

In [8]:
def process_query(query):
    start = time.time()
    print('QUESTION:', query)
    keywords = find_keywords(query)
#     print(keywords)
    with driver.session() as session:
        sentences = session.read_transaction(shortlist_sentences, keywords)[:10]
        for sentence in sentences:
            if '.' not in sentence[1][-2:]:
                sentence[1] += '.'
#             print('*', '(' + str(sentence[2]) + ')', sentence[1])
#             print(sentence[1] + '.')
    passage = '\n'.join(sentence[1] for sentence in sentences)
    answers = mrc.predict(query, passage)["best_span_str"].split('.')
    print('ANSWER:')
    for answer in answers:
        for sentence in sentences:
            if answer in sentence[1]:
                print(sentence[1])
    end = time.time()
    print('Time taken:', round(100 * (end - start))/100, 'secs')
    print('-' * 50)

def shortlist_sentences(tx, keywords):
    query = (
        'call { \n' +
        'with \n' + str(keywords) + ' as keywords \n' +
        'match (p:Paragraph)-[]->(s:Sentence)-[]->(:Extraction)-[]->(e:ExtEntity) \n' + 
        'match (p)-[]->(:Sentence)-[]->(:Extraction)-[]->(e_nbr:ExtEntity) \n' + 
        'with collect(distinct e) as entities, collect(distinct e_nbr) as para_entities, s, keywords \n' + 
        'return s.id as s_id, s.text as sentence, round(10 * reduce( \n' + 
            'total = 0.0, keyword in keywords| \n' + 
                'total + reduce( \n' + 
                    'isPresent = 0.0, entity in entities | \n' + 
                    'case \n' + 
                        'when entity.text = keyword[0] and isPresent < 1.0 then 1.0 \n' + 
                        'when entity.text starts with keyword[1] and isPresent < 0.7 then 0.7 \n' +
                        'else isPresent \n' + 
                    'end \n' + 
                ') + reduce( \n' + 
                    'isPresent = 0.0, entity in para_entities | \n' + 
                    'case \n' + 
                        'when entity in entities then isPresent \n' + 
                        'when entity.text = keyword[0] and isPresent < 0.2 then 0.2 \n' + 
                        'when entity.text starts with keyword[1] and isPresent < 0.1 then 0.1 \n' + 
                        'else isPresent \n' + 
                    'end \n' + 
                ') \n' + 
            ')\n' + 
        ')/10 as score \n' +
        'union \n' +
        'with \n' + str(keywords) + ' as keywords \n' +
        'match (p:Paragraph)-[]->(s:Sentence)-[:about_entity]->(e:ExtEntity) \n' + 
        'with collect(distinct e) as entities, s, keywords \n' + 
        'return s.id as s_id, s.text as sentence, round(10 * reduce( \n' + 
            'total = 0.0, keyword in keywords| \n' + 
                'total + reduce( \n' + 
                    'isPresent = 0.0, entity in entities | \n' + 
                    'case \n' + 
                        'when entity.text = keyword[0] and isPresent < 1.0 then 1.0 \n' + 
                        'when entity.text starts with keyword[1] and isPresent < 0.7 then 0.7 \n' +
                        'else isPresent \n' + 
                    'end \n' + 
                ') \n' + 
            ')\n' + 
        ')/10 as score \n' + 
        '} \n' + 
        'with s_id, sentence, score \n' + 
        'where score >= 0.7 \n' + 
        'return s_id, sentence, score \n' + 
        'order by score desc '
    )
    result = tx.run(query)
    sentences = []
    for record in result:
        sentences.append([record['s_id'], record['sentence'], record['score']])
    return sentences

def get_para_of_sentence(sid):
    query = ('match (p:Paragraph)-->(s:Sentence) where s.id = ' + str(s_id) + 'return p.id, p.text')

In [9]:
process_query('What is the normal load for UG students')
process_query('If I fail a course and take it again in the later semester, will my earlier course with F grade be removed from the transcript')
process_query('what is the process of registration?')
process_query('how many seats are there in cse for admission?')
process_query('what is the admission criteria for btech')
process_query('I am in 1st year. Can I take overload?')
process_query('I am in 2nd year. Can I take overload?')
process_query('what happens if I miss the endsem because of a medical reason?')
process_query('what happens if I fail a course?')
process_query('what happens if I get an F grade in a course?')

QUESTION: What is the normal load for UG students
ANSWER:
Normal load for the first and second year students is 16-20 credits and for the third and fourth year students is 16-22 credits.
Time taken: 8.61 secs
--------------------------------------------------
QUESTION: If I fail a course and take it again in the later semester, will my earlier course with F grade be removed from the transcript
ANSWER:
The pass grade for the respective course will count towards the SGPA of the semester in which the course is passed and not in the semester where F’ grade was awarded.
Time taken: 10.96 secs
--------------------------------------------------
QUESTION: what is the process of registration?
ANSWER:
Registration is compulsory for all students who are not on leave of absence, and is the sole responsibility of the student and must be completed before the last date of registration.
Time taken: 9.41 secs
--------------------------------------------------
QUESTION: how many seats are there in cse f